In [1]:
from read_data import df_customer, df_category, df_product, df_receipt, df_store, df_geocode

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [3]:
# 001
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


In [4]:
# 002
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


In [5]:
# 003
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns={'sales_ymd': 'sales_date'}).head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


In [6]:
# 004
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [7]:
# 004-2
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']][df_receipt['customer_id'] == 'CS018205000001']

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [8]:
# 005
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and amount >= 1000')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [9]:
# 006
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].query('customer_id == "CS018205000001" and (amount >= 1000 or quantity >= 5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


In [10]:
# 007
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and (1000 <= amount <= 2000)')

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


In [11]:
# 008
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and product_cd != "P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [12]:
# 009
df_store.query('prefecture_cd != "13" and floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


In [13]:
# 010
df_store.query('store_cd.str.startswith("S14")').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


In [14]:
# 011
df_customer.query('customer_id.str.endswith("1")').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


In [15]:
# 012
df_store.query('address.str.contains("横浜市")')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


In [16]:
# 013
df_customer.query('status_cd.str.contains(r"^[A-F]", regex=True)').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


In [17]:
# 014
df_customer.query('status_cd.str.contains(r"[1-9]$", regex=True)').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


In [18]:
# 015
df_customer.query('status_cd.str.contains(r"^[A-F].*[1-9]$", regex=True)').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


In [19]:
# 016
df_store.query('tel_no.str.contains(r"^\d\d\d-\d\d\d-\d\d\d\d$", regex=True)').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


In [20]:
# 017
df_customer.sort_values('birth_day').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


In [21]:
# 018
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


In [22]:
# 019
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']],
                    df_receipt['amount'].rank(method='min', ascending=False)], axis=1)

df_tmp.columns = ['customer_id', 'amount', 'ranking']

df_tmp.sort_values('ranking').head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


In [23]:
# 020
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']],
                    df_receipt['amount'].rank(method='first', ascending=False)], axis=1)

df_tmp.columns = ['customer_id', 'amount', 'ranking']

df_tmp.sort_values('ranking').head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


In [24]:
# 021
len(df_receipt)

104681

In [25]:
# 022
len(df_receipt['customer_id'].unique())

8307

In [26]:
# 023
df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


In [27]:
# 024
df_receipt.groupby('customer_id').agg({'sales_ymd': 'max'}).reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


In [28]:
# 025
df_receipt.groupby('customer_id').agg({'sales_ymd': 'min'}).reset_index().head(10)
#df_receipt.groupby('customer_id').sales_ymd.min().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


In [29]:
# 026
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd': ['min', 'max']}).reset_index()
df_tmp.columns = ['customer_id', 'sales_ymd_min', 'sales_ymd_max']
df_tmp.query('sales_ymd_min != sales_ymd_max').head(10)

,customer_id,sales_ymd_min,sales_ymd_max
1,CS001114000005,20180503,20190731
2,CS001115000010,20171228,20190405
3,CS001205000004,20170914,20190625
4,CS001205000006,20180207,20190224
13,CS001214000009,20170306,20190902
14,CS001214000017,20180828,20191006
16,CS001214000048,20171109,20190929
17,CS001214000052,20180208,20190617
20,CS001215000005,20170206,20181021
21,CS001215000040,20170214,20171022


In [30]:
# 027
df_receipt.groupby('store_cd').amount.mean().reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


In [31]:
# 028
df_receipt.groupby('store_cd').amount.median().reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


In [32]:
# 029
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index()[['store_cd','product_cd']].sort_values('store_cd').head(10)


,store_cd,product_cd
0,S12007,P060303001
1,S12013,P060303001
2,S12014,P060303001
3,S12029,P060303001
4,S12030,P060303001
5,S13001,P060303001
6,S13002,P060303001
7,S13003,P071401001
8,S13004,P060303001
9,S13005,P040503001


In [33]:
# 030
df_receipt.groupby('store_cd').amount.var(ddof=0).reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


In [34]:
# 031
df_receipt.groupby('store_cd').amount.std(ddof=0).reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


In [35]:
# 032
df_receipt.amount.quantile([0.25, 0.5, 0.75, 1.0]).reset_index()

,index,amount
0,0.25,102.0
1,0.50,170.0
2,0.75,288.0
3,1.00,10925.0


In [36]:
# 033
df_receipt.groupby('store_cd').amount.mean().reset_index().query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


In [37]:
# 034
df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().reset_index().amount.mean()

2547.742234529256

In [38]:
# 035
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().reset_index()
df_tmp.amount.mean()
df_tmp.query('amount >= {}'.format(df_tmp.amount.mean())).head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


In [39]:
# 036
pd.merge(df_receipt, df_store[['store_cd', 'store_name']], how='inner', on='store_cd').head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


In [40]:
# 037
pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']],
                                how='inner', on='category_small_cd').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


In [41]:
# 038
df_r = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().reset_index()
df_c = df_customer.query('gender_cd == "1"')
pd.merge(df_c, df_r, how='left', on='customer_id').fillna(0)[['customer_id', 'amount']].head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


In [42]:
# 039
df_tmp1 = df_receipt[~df_receipt.duplicated(subset=['customer_id', 'sales_ymd'])].query('not customer_id.str.startswith("Z")').groupby('customer_id').sales_ymd.count().reset_index().sort_values('sales_ymd', ascending=False).head(20)
df_tmp2 = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().reset_index().sort_values('amount', ascending=False).head(20)

pd.merge(df_tmp1, df_tmp2, how='outer', on='customer_id').fillna(0)


,customer_id,sales_ymd,amount
0,CS040214000008,23.0,0.0
1,CS015415000185,22.0,20153.0
2,CS010214000010,22.0,18585.0
3,CS010214000002,21.0,0.0
4,CS028415000007,21.0,19127.0
5,CS017415000097,20.0,23086.0
6,CS016415000141,20.0,18372.0
7,CS031414000051,19.0,19202.0
8,CS014214000023,19.0,0.0
9,CS022515000226,19.0,0.0


In [43]:
# 040
len(pd.merge(df_store, df_product, how='cross'))

531590

In [44]:
# 041
df_tmp = df_receipt.groupby('sales_ymd').amount.sum()
df_tmp2 = df_tmp.copy().shift()
df_tmp = pd.concat([df_tmp, df_tmp2], axis=1).reset_index()
df_tmp.columns = ['sales_ymd', 'amount1', 'amount2']
df_tmp['lift'] = df_tmp['amount1'] - df_tmp['amount2']
df_tmp.head(10)

,sales_ymd,amount1,amount2,lift
0,20170101,33723,NaN,NaN
1,20170102,24165,33723.0,-9558.0
2,20170103,27503,24165.0,3338.0
3,20170104,36165,27503.0,8662.0
4,20170105,37830,36165.0,1665.0
5,20170106,32387,37830.0,-5443.0
6,20170107,23415,32387.0,-8972.0
7,20170108,24737,23415.0,1322.0
8,20170109,26718,24737.0,1981.0
9,20170110,20143,26718.0,-6575.0


In [45]:
# 042
df_tmp = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_tmp2 = df_tmp.copy().shift()
df_tmp3 = df_tmp2.copy().shift()
df_tmp4 = df_tmp3.copy().shift()
df_tmp = pd.concat([df_tmp, df_tmp2, df_tmp3, df_tmp4], axis=1).copy()
df_tmp.columns = ['sales_ymd', 'amount', 'sales_ymd1', 'amount1', 'sales_ymd2', 'amount2', 'sales_ymd3', 'amount3']
df_tmp.head(10)

,sales_ymd,amount,sales_ymd1,amount1,sales_ymd2,amount2,sales_ymd3,amount3
0,20170101,33723,NaN,NaN,NaN,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,NaN,NaN,NaN,NaN
2,20170103,27503,20170102.0,24165.0,20170101.0,33723.0,NaN,NaN
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0


In [46]:
# 043
df_tmp = pd.merge(df_receipt, df_customer, how='inner', on='customer_id').copy()
df_tmp['age2'] = df_tmp.age.apply(lambda x: math.floor(x/10)*10)
#df_tmp = df_tmp[['age2', 'gender_cd', 'amount']].groupby(['gender_cd', 'age2']).amount.sum().reset_index()
#df_tmp2 = pd.merge(df_tmp.query('gender_cd == "0"'), df_tmp.query('gender_cd == "1"'), how='outer', on='age2')
#df_tmp2 = pd.merge(df_tmp2, df_tmp.query('gender_cd == "9"'), how='outer', on='age2')
#df_tmp = df_tmp2[['age2', 'amount_x', 'amount_y', 'amount']]

df_tmp = pd.pivot_table(df_tmp, index='age2', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
df_tmp.columns = ['age', 'amount_men', 'amount_women', 'amount_unknown']
df_tmp

,age,amount_men,amount_women,amount_unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


In [47]:
# 044
df_tmp = pd.merge(df_receipt, df_customer, how='inner', on='customer_id').copy()
df_tmp['age2'] = df_tmp.age.apply(lambda x: math.floor(x/10)*10)
df_tmp = pd.pivot_table(df_tmp, index='age2', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
df_tmp.columns = ['age', 'amount_men', 'amount_women', 'amount_unknown']

df_tmp = df_tmp.set_index('age').stack().reset_index().replace({'amount_women': '01', 'amount_men': '00', 'amount_unknown': '99'}).rename(columns={'level_1': 'gender_cd', 0: 'amount'})

#df_tmp00 = df_tmp[['age', 'amount_men']]
#df_tmp00['gender_cd'] = '00'
#df_tmp01 = df_tmp[['age', 'amount_women']]
#df_tmp01['gender_cd'] = '01'
#df_tmp99 = df_tmp[['age', 'amount_unknown']]
#df_tmp99['gender_cd'] = '99'
#df_tmp00.columns = ['age', 'amount', 'gender_cd']
#df_tmp01.columns = ['age', 'amount', 'gender_cd']
#df_tmp99.columns = ['age', 'amount', 'gender_cd']
#df_tmp = pd.concat([df_tmp00, df_tmp01, df_tmp99], axis=0)
df_tmp[['age', 'gender_cd', 'amount']]

,age,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


In [48]:
# 045
df_tmp = df_customer[['customer_id', 'birth_day']].copy()
df_tmp['birthday'] = df_customer.birth_day.apply(lambda x: "{:04d}{:02d}{:02d}".format(x.year, x.month, x.day))
df_tmp[['customer_id', 'birthday']]

,customer_id,birthday
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
...,...,...
21966,CS002512000474,19591012
21967,CS029414000065,19701019
21968,CS012403000043,19721216
21969,CS033512000184,19640605


In [49]:
# 046
df_tmp = df_customer[['customer_id', 'application_date']]
df_tmp['application_date'] = df_tmp.application_date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))
df_tmp.head(10)

<ipython-input-49-2c4e41e6df6b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['application_date'] = df_tmp.application_date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))


,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


In [50]:
# 047
df_tmp = df_receipt.copy()
#df_tmp['sales_ymd'] = df_receipt.sales_ymd.apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp.sales_ymd.astype('str'))
df_tmp[['sales_ymd', 'receipt_no', 'receipt_sub_no']].head(10)

,sales_ymd,receipt_no,receipt_sub_no
0,2018-11-03,112,1
1,2018-11-18,1132,2
2,2017-07-12,1102,1
3,2019-02-05,1132,1
4,2018-08-21,1102,2
5,2019-06-05,1112,1
6,2018-12-05,1102,2
7,2019-09-22,1102,1
8,2017-05-04,1112,2
9,2019-10-10,1102,1


In [51]:
# 048
df_tmp = df_receipt.copy()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_epoch'], unit='s')
df_tmp[['receipt_no', 'receipt_sub_no', 'sales_ymd']].head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


In [52]:
# 049
df_tmp = df_receipt.copy()
#df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_epoch'], unit='s')
#df_tmp['sales_year'] = df_tmp.sales_ymd.apply(lambda x: str(x)[0:4])
df_tmp['sales_year'] = pd.to_datetime(df_tmp['sales_epoch'], unit='s').dt.year

df_tmp[['sales_year', 'receipt_no', 'receipt_sub_no']].head(10)

,sales_year,receipt_no,receipt_sub_no
0,2018,112,1
1,2018,1132,2
2,2017,1102,1
3,2019,1132,1
4,2018,1102,2
5,2019,1112,1
6,2018,1102,2
7,2019,1102,1
8,2017,1112,2
9,2019,1102,1


In [53]:
# 050
df_tmp = df_receipt.copy()
#df_tmp['sales_month'] = pd.to_datetime(df_tmp['sales_epoch'], unit='s').dt.month.apply(lambda x: "{:02d}".format(x))
df_tmp['sales_month'] = pd.to_datetime(df_tmp['sales_epoch'], unit='s').dt.strftime('%m')
df_tmp[['sales_month', 'receipt_no', 'receipt_sub_no']].head(10)

,sales_month,receipt_no,receipt_sub_no
0,11,112,1
1,11,1132,2
2,07,1102,1
3,02,1132,1
4,08,1102,2
5,06,1112,1
6,12,1102,2
7,09,1102,1
8,05,1112,2
9,10,1102,1


In [54]:
# 051
df_tmp = df_receipt.copy()
df_tmp['sales_day'] = pd.to_datetime(df_tmp['sales_epoch'], unit='s').dt.strftime('%d')
df_tmp[['receipt_no', 'receipt_sub_no', 'sales_day']].head(10)

,receipt_no,receipt_sub_no,sales_day
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


In [55]:
# 052
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().reset_index().copy()
#df_tmp1 = df_tmp.query('amount > 2000').copy()
#df_tmp1['flag'] = 1
#df_tmp2 = df_tmp.query('amount <= 2000').copy()
#df_tmp2['flag'] = 0
#df_tmp = pd.concat([df_tmp1, df_tmp2])
df_tmp['flag'] = np.where(df_tmp['amount'] > 2000, 1, 0)
df_tmp.sort_values('customer_id').head(10)

,customer_id,amount,flag
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


In [56]:
# 053
df_tmp = df_customer.copy()
#df_tmp['postal_flag'] = df_customer['postal_cd'].str[0:3].apply(lambda x: int(x))
#df_tmp1 = df_tmp.query('100 <= postal_flag <= 209')
#df_tmp2 = df_tmp.query('not (100 <= postal_flag <= 209)')
#df_tmp1['postal_flag'] = 1
#df_tmp2['postal_flag'] = 0
#df_tmp = pd.concat([df_tmp1, df_tmp2]).sort_values('customer_id')
df_tmp['postal_flag'] = df_tmp['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)

df_tmp = pd.merge(df_receipt, df_tmp, how='inner', on='customer_id')
df_tmp.groupby(['postal_flag']).agg({'customer_id': 'nunique'}).reset_index()

,postal_flag,customer_id
0,0,3906
1,1,4400


In [57]:
# 054
df_tmp = df_customer.copy()
df_tmp['pref_code'] = df_tmp['address'].str[0:3].map({'埼玉県': 11, '千葉県': 12, '東京都': 13, '神奈川': 14})
df_tmp.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,pref_code
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,14
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,13
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,13
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,14
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,13


In [58]:
# 055
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().reset_index().sort_values('amount').copy()
pct = df_tmp['amount'].quantile([0.25, 0.5, 0.75, 1.0])
#print(pct)
df_tmp1 = df_tmp.query('amount <= {}'.format(pct[0.25])).copy()
df_tmp1['amount_flag'] = 1
df_tmp2 = df_tmp.query('{} < amount <= {}'.format(pct[0.25], pct[0.5])).copy()
df_tmp2['amount_flag'] = 2
df_tmp3 = df_tmp.query('{} < amount <= {}'.format(pct[0.5], pct[0.75])).copy()
df_tmp3['amount_flag'] = 3
df_tmp4 = df_tmp.query('{} < amount'.format(pct[0.75])).copy()
df_tmp4['amount_flag'] = 4
pd.concat([df_tmp1, df_tmp2, df_tmp3, df_tmp4]).sort_values('customer_id').head(10)


,customer_id,amount,amount_flag
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


In [59]:
# 056
df_tmp = df_customer.copy()
df_tmp['age_group'] = df_tmp.age.apply(lambda x: x - x%10)
df_tmp['age_group'] = df_tmp.age_group.apply(lambda x: 60 if x >= 60 else x)
df_tmp[['customer_id', 'birth_day', 'age_group']].head(10)

,customer_id,birth_day,age_group
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


In [60]:
# 057
df_tmp = df_customer.copy()
df_tmp['age_group'] = df_tmp.age.apply(lambda x: x - x%10)
df_tmp['age_group'] = df_tmp.age_group.apply(lambda x: 60 if x >= 60 else x)
df_tmp['gender_age'] = df_tmp['gender_cd'] + '_' + df_tmp['age_group'].astype('str')
df_tmp.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,age_group,gender_age
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,30,1_30
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,60,9_60
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,40,1_40
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,60,1_60
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,20,1_20
5,CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0,40,0_40
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,40,1_40
7,CS029403000008,釈 人志,0,男性,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0,40,0_40
8,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0,60,0_60
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5,50,1_50


In [61]:
# 058
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


In [62]:
# 059
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().copy().reset_index()
df_tmp['std_amount'] = preprocessing.scale(df_tmp['amount'])
df_tmp.head(10)

,customer_id,amount,std_amount
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


In [63]:
# 060
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")').groupby('customer_id').amount.sum().copy().reset_index()
df_tmp['scale_amount'] = preprocessing.minmax_scale(df_tmp['amount'])
df_tmp.head(10)

,customer_id,amount,scale_amount
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


In [64]:
# 061
df_tmp = df_receipt.query("not customer_id.str.startswith('Z')").groupby('customer_id').amount.sum().reset_index()
df_tmp['log_amount'] = np.log10(df_tmp['amount'] + 0.5)
df_tmp.head(10)

,customer_id,amount,log_amount
0,CS001113000004,1298,3.113442
1,CS001114000005,626,2.796921
2,CS001115000010,3044,3.483516
3,CS001205000004,1988,3.298526
4,CS001205000006,3337,3.523421
5,CS001211000025,456,2.659441
6,CS001212000027,448,2.651762
7,CS001212000031,296,2.472025
8,CS001212000046,228,2.358886
9,CS001212000070,456,2.659441


In [65]:
# 062
df_tmp = df_receipt.query("not customer_id.str.startswith('Z')").groupby('customer_id').amount.sum().reset_index()
df_tmp['log_amount'] = np.log(df_tmp['amount'] + 0.5)
df_tmp.head(10)

,customer_id,amount,log_amount
0,CS001113000004,1298,7.168965
1,CS001114000005,626,6.440149
2,CS001115000010,3044,8.021092
3,CS001205000004,1988,7.595136
4,CS001205000006,3337,8.112977
5,CS001211000025,456,6.123589
6,CS001212000027,448,6.105909
7,CS001212000031,296,5.692047
8,CS001212000046,228,5.431536
9,CS001212000070,456,6.123589


In [66]:
# 063
df_tmp = df_product.copy()
df_tmp['unit_mergin'] = df_tmp['unit_price'] - df_tmp['unit_cost']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_mergin
0,P040101001,04,0401,040101,198.0,149.0,49.0
1,P040101002,04,0401,040101,218.0,164.0,54.0
2,P040101003,04,0401,040101,230.0,173.0,57.0
3,P040101004,04,0401,040101,248.0,186.0,62.0
4,P040101005,04,0401,040101,268.0,201.0,67.0
5,P040101006,04,0401,040101,298.0,224.0,74.0
6,P040101007,04,0401,040101,338.0,254.0,84.0
7,P040101008,04,0401,040101,420.0,315.0,105.0
8,P040101009,04,0401,040101,498.0,374.0,124.0
9,P040101010,04,0401,040101,580.0,435.0,145.0


In [67]:
# 064
df_tmp = df_product.copy()
df_tmp['unit_profit'] = (df_tmp['unit_price'] - df_tmp['unit_cost']) / df_tmp['unit_price']
df_tmp.unit_profit.mean()

0.24911389885177

In [68]:
# 065
df_tmp = df_product.copy()
df_tmp['unit_price_new'] = df_tmp['unit_cost'] / 7 * 10
df_tmp['unit_price_new'] = df_tmp.query('unit_price_new >= 0').unit_price_new.apply(lambda x: math.floor(x*10)/10)
df_tmp['unit_profit_new'] = (df_tmp['unit_price_new'] - df_tmp['unit_cost']) / df_tmp['unit_price_new']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_new,unit_profit_new
0,P040101001,04,0401,040101,198.0,149.0,212.8,0.299812
1,P040101002,04,0401,040101,218.0,164.0,234.2,0.299744
2,P040101003,04,0401,040101,230.0,173.0,247.1,0.299879
3,P040101004,04,0401,040101,248.0,186.0,265.7,0.299962
4,P040101005,04,0401,040101,268.0,201.0,287.1,0.299896
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,362.8,0.299890
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.2,0.299888
9,P040101010,04,0401,040101,580.0,435.0,621.4,0.299968


In [69]:
# 066
df_tmp = df_product.copy()
df_tmp['unit_price_new'] = np.round(df_tmp['unit_cost'] / 7 * 10)
df_tmp['unit_profit_new'] = (df_tmp['unit_price_new'] - df_tmp['unit_cost']) / df_tmp['unit_price_new']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_new,unit_profit_new
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


In [70]:
# 067
df_tmp = df_product.copy()
df_tmp['unit_price_new'] = np.ceil(df_tmp['unit_cost'] / 7 * 10)
df_tmp['unit_profit_new'] = (df_tmp['unit_price_new'] - df_tmp['unit_cost']) / df_tmp['unit_price_new']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_new,unit_profit_new
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,235.0,0.302128
2,P040101003,04,0401,040101,230.0,173.0,248.0,0.302419
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,288.0,0.302083
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,535.0,0.300935
9,P040101010,04,0401,040101,580.0,435.0,622.0,0.300643


In [71]:
# 068
df_tmp = df_product.copy()
df_tmp['unit_price_tax'] = np.floor(df_tmp['unit_price'] * 1.1)
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_tax
0,P040101001,04,0401,040101,198.0,149.0,217.0
1,P040101002,04,0401,040101,218.0,164.0,239.0
2,P040101003,04,0401,040101,230.0,173.0,253.0
3,P040101004,04,0401,040101,248.0,186.0,272.0
4,P040101005,04,0401,040101,268.0,201.0,294.0
5,P040101006,04,0401,040101,298.0,224.0,327.0
6,P040101007,04,0401,040101,338.0,254.0,371.0
7,P040101008,04,0401,040101,420.0,315.0,462.0
8,P040101009,04,0401,040101,498.0,374.0,547.0
9,P040101010,04,0401,040101,580.0,435.0,638.0


In [72]:
# 069
df_tmp = pd.merge(df_receipt, df_product,
                  how='inner', on='product_cd').copy()
df_tmp1 = df_tmp.groupby('customer_id').amount.sum().reset_index().copy()
df_tmp2 = df_tmp.query('category_major_cd == "07"').groupby('customer_id').amount.sum().reset_index().copy()
df_tmp = pd.merge(df_tmp1, df_tmp2,
                  how='inner', on='customer_id').copy()
df_tmp['ratio'] = df_tmp['amount_y'] / df_tmp['amount_x']
df_tmp.head(10)

,customer_id,amount_x,amount_y,ratio
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


In [73]:
# 070
df_tmp = pd.merge(df_receipt, df_customer,
                  how='inner', on='customer_id').copy()

df_tmp['sales_ymd_2'] = df_tmp['sales_ymd'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
df_tmp['application_date_2'] = df_tmp['application_date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))

df_tmp['date_diff'] = (df_tmp['sales_ymd_2'] - df_tmp['application_date_2']).dt.days
df_tmp[['customer_id', 'sales_ymd', 'application_date', 'date_diff']].head(10)

,customer_id,sales_ymd,application_date,date_diff
0,CS006214000001,20181103,20150201,1371
1,CS006214000001,20170509,20150201,828
2,CS006214000001,20170608,20150201,858
3,CS006214000001,20170608,20150201,858
4,CS006214000001,20181028,20150201,1365
5,CS006214000001,20181028,20150201,1365
6,CS006214000001,20170509,20150201,828
7,CS006214000001,20190908,20150201,1680
8,CS006214000001,20180131,20150201,1095
9,CS006214000001,20170705,20150201,885


In [74]:
# 071
df_tmp = pd.merge(df_receipt, df_customer, how='inner', on='customer_id').copy()
df_tmp['sales_ymd_2'] = pd.to_datetime(df_tmp.sales_ymd.astype('str'))
df_tmp['application_date_2'] = pd.to_datetime(df_tmp.application_date)
df_tmp['elapsed_months'] = (df_tmp.sales_ymd_2.apply(lambda x: x.year) - df_tmp.application_date_2.apply(lambda x: x.year)) * 12 + (df_tmp.sales_ymd_2.apply(lambda x: x.month) - df_tmp.application_date_2.apply(lambda x: x.month))
df_tmp[['customer_id', 'sales_ymd', 'application_date', 'elapsed_months']].head(10)

,customer_id,sales_ymd,application_date,elapsed_months
0,CS006214000001,20181103,20150201,45
1,CS006214000001,20170509,20150201,27
2,CS006214000001,20170608,20150201,28
3,CS006214000001,20170608,20150201,28
4,CS006214000001,20181028,20150201,44
5,CS006214000001,20181028,20150201,44
6,CS006214000001,20170509,20150201,27
7,CS006214000001,20190908,20150201,55
8,CS006214000001,20180131,20150201,35
9,CS006214000001,20170705,20150201,29


In [75]:
# 072
df_tmp = pd.merge(df_receipt, df_customer, how='inner', on='customer_id').copy()
df_tmp['sales_ymd_2'] = pd.to_datetime(df_tmp.sales_ymd.astype('str'))
df_tmp['application_date_2'] = pd.to_datetime(df_tmp.application_date)
df_tmp['elapsed_years'] = df_tmp.sales_ymd_2.apply(lambda x: x.year) - df_tmp.application_date_2.apply(lambda x: x.year)
df_tmp[['customer_id', 'sales_ymd', 'application_date', 'elapsed_years']].head(10)

,customer_id,sales_ymd,application_date,elapsed_years
0,CS006214000001,20181103,20150201,3
1,CS006214000001,20170509,20150201,2
2,CS006214000001,20170608,20150201,2
3,CS006214000001,20170608,20150201,2
4,CS006214000001,20181028,20150201,3
5,CS006214000001,20181028,20150201,3
6,CS006214000001,20170509,20150201,2
7,CS006214000001,20190908,20150201,4
8,CS006214000001,20180131,20150201,3
9,CS006214000001,20170705,20150201,2


In [76]:
# 073
df_tmp = pd.merge(df_receipt, df_customer, how='inner', on='customer_id').copy()
df_tmp['sales_ymd_epoch'] = pd.to_datetime(df_tmp.sales_ymd.astype('str')).dt.strftime('%s').astype('int')
df_tmp['application_date_epoch'] = pd.to_datetime(df_tmp.application_date).dt.strftime('%s').astype('int')
df_tmp['elapsed_seconds'] = df_tmp.sales_ymd_epoch - df_tmp.application_date_epoch
df_tmp[['customer_id', 'sales_ymd', 'application_date', 'elapsed_seconds']].head(10)

,customer_id,sales_ymd,application_date,elapsed_seconds
0,CS006214000001,20181103,20150201,118454400
1,CS006214000001,20170509,20150201,71539200
2,CS006214000001,20170608,20150201,74131200
3,CS006214000001,20170608,20150201,74131200
4,CS006214000001,20181028,20150201,117936000
5,CS006214000001,20181028,20150201,117936000
6,CS006214000001,20170509,20150201,71539200
7,CS006214000001,20190908,20150201,145152000
8,CS006214000001,20180131,20150201,94608000
9,CS006214000001,20170705,20150201,76464000


In [77]:
# 074
df_tmp = pd.merge(df_receipt, df_customer, how='inner', on='customer_id').copy()
df_tmp['sales_ymd_wday'] = pd.to_datetime(df_tmp.sales_ymd.astype('str')).apply(lambda x: x.weekday())
df_tmp['sales_ymd_monday'] = pd.to_datetime(df_tmp.sales_ymd.astype('str')).apply(lambda x: x - relativedelta(days=x.weekday()))
#df_tmp['sales_ymd_wday'] = pd.to_datetime(df_tmp.sales_ymd.astype('str')).dt.strftime('%u').astype('int').apply(lambda x: x - 1)
#df_tmp['sales_ymd_monday'] = pd.to_datetime(df_tmp.sales_ymd.astype('str')) + df_tmp['sales_ymd_wday'].apply(lambda x: timedelta(days=-1 * x))
df_tmp[['customer_id', 'sales_ymd', 'sales_ymd_wday', 'sales_ymd_monday']].head(10)

,customer_id,sales_ymd,sales_ymd_wday,sales_ymd_monday
0,CS006214000001,20181103,5,2018-10-29
1,CS006214000001,20170509,1,2017-05-08
2,CS006214000001,20170608,3,2017-06-05
3,CS006214000001,20170608,3,2017-06-05
4,CS006214000001,20181028,6,2018-10-22
5,CS006214000001,20181028,6,2018-10-22
6,CS006214000001,20170509,1,2017-05-08
7,CS006214000001,20190908,6,2019-09-02
8,CS006214000001,20180131,2,2018-01-29
9,CS006214000001,20170705,2,2017-07-03


In [78]:
# 075
#n = len(df_customer) * 0.01
#df_customer.sample(n=int(n)).head(10)
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18963,CS021513000120,堀越 真悠子,1,女性,1964-09-22,54,259-1143,神奈川県伊勢原市下糟屋**********,S14021,20151103,5-20100221-5
12991,CS005412000060,本間 朝陽,1,女性,1970-09-27,48,167-0022,東京都杉並区下井草**********,S13005,20150217,0-00000000-0
1569,CS026513000076,小平 遥,1,女性,1959-09-20,59,251-0045,神奈川県藤沢市辻堂東海岸**********,S14026,20150714,0-00000000-0
16592,CS002312000305,野際 礼子,1,女性,1986-02-13,33,187-0002,東京都小平市花小金井**********,S13002,20170927,0-00000000-0
21775,CS020512000053,三宅 知世,1,女性,1964-09-18,54,173-0004,東京都板橋区板橋**********,S13020,20150129,2-20091101-3
7496,CS005603000073,西岡 栄一,0,男性,1950-11-26,68,167-0033,東京都杉並区清水**********,S13005,20170127,0-00000000-0
7712,CS005504000014,黒川 直人,0,男性,1961-06-13,57,167-0022,東京都杉並区下井草**********,S13005,20150831,0-00000000-0
1757,CS011312000092,島崎 夏空,1,女性,1985-07-27,33,212-0058,神奈川県川崎市幸区鹿島田**********,S14011,20151003,0-00000000-0
6145,CS003613000173,杉山 育子,1,女性,1955-07-06,63,201-0012,東京都狛江市中和泉**********,S13003,20160518,0-00000000-0
7469,CS004414000262,金城 なつみ,1,女性,1971-02-23,48,165-0035,東京都中野区白鷺**********,S13004,20160128,9-20100722-A


In [79]:
# 076
#df_tmp = pd.concat([df_customer.query('gender_cd == "0"').sample(frac=0.1),
#                    df_customer.query('gender_cd == "1"').sample(frac=0.1),
#                   df_customer.query('gender_cd == "9"').sample(frac=0.1)]).copy()
_, df_tmp = train_test_split(df_customer, test_size=0.1, stratify=df_customer['gender_cd'])

df_tmp[['customer_id', 'gender_cd']].groupby('gender_cd').count()

,customer_id
gender_cd,
0,298
1,1793
9,107


In [80]:
# 077
df_tmp = df_receipt[['customer_id', 'amount']].groupby('customer_id').agg({'amount': 'sum'}).copy()
df_tmp['log_amount'] = np.log10(df_tmp['amount'])

mean = df_tmp.log_amount.agg({'log_amount': 'mean'})
std = df_tmp.log_amount.agg({'log_amount': 'std'})

upper = mean +  3 * std
lower = mean - 3 * std

df_tmp.query('log_amount > {} or log_amount < {}'.format(float(upper), float(lower)))

,amount,log_amount
customer_id,,
ZZ000000000000,12395003,7.093247


In [81]:
# 078
df_tmp = df_receipt[['customer_id', 'amount']].query('not customer_id.str.startswith("Z")').groupby('customer_id').agg({'amount': 'sum'}).copy()
p25 = np.percentile(df_tmp.amount, q=25)
p75 = np.percentile(df_tmp.amount, q=75)
iqr = p75 - p25
upper = p75 + 1.5 * iqr
lower = p25 - 1.5 * iqr
#print(lower, p25, iqr, p75, upper)
df_tmp.query('amount > @upper or amount < @lower').head(10)

,amount
customer_id,
CS001414000048,8584
CS001605000009,18925
CS002415000594,9568
CS004414000181,9584
CS005415000137,8734
CS006414000001,9156
CS006414000029,9179
CS006415000105,10042
CS006415000147,12723


In [82]:
# 079
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

In [83]:
# 080
df_product_1 = df_product.copy()
df_product_1.dropna(inplace=True)

print(len(df_product))
print(len(df_product_1))

10030
10023
